<a href="https://colab.research.google.com/github/buganart/unagan/blob/master/Unagan_generate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title Setup
# @markdown 1. Before starting please save the notebook in your drive by clicking on `File -> Save a copy in drive`
# @markdown 2. Check GPU, should be a Tesla V100 if you want to train it as fast as possible.
# @markdown 3. Mount google drive.
# @markdown 4. Log in to wandb.

!nvidia-smi -L
import os
os.environ["WANDB_MODE"] = "dryrun"

print(f"We have {os.cpu_count()} CPU cores.")
print()

try:
    from google.colab import drive, output

    IN_COLAB = True
except ImportError:
    from IPython.display import clear_output

    IN_COLAB = False

from pathlib import Path

if IN_COLAB:
    drive.mount("/content/drive/")

    if not Path("/content/drive/My Drive/IRCMS_GAN_collaborative_database").exists():
        raise RuntimeError(
            "Shortcut to our shared drive folder doesn't exits.\n\n"
            "\t1. Go to the google drive web UI\n"
            '\t2. Right click shared folder IRCMS_GAN_collaborative_database and click "Add shortcut to Drive"'
        )

clear = output.clear if IN_COLAB else clear_output

def clear_on_success(msg="Ok!"):
    if _exit_code == 0:
        clear()
        print(msg)

print()
print("Wandb installation and login ...")
%pip install -q wandb

wandb_drive_netrc_path = Path("drive/My Drive/colab/.netrc")
wandb_local_netrc_path = Path("/root/.netrc")
if wandb_drive_netrc_path.exists():
    import shutil

    print("Wandb .netrc file found, will use that to log in.")
    shutil.copy(wandb_drive_netrc_path, wandb_local_netrc_path)
else:
    print(
        f"Wandb config not found at {wandb_drive_netrc_path}.\n"
        f"Using manual login.\n\n"
        f"To use auto login in the future, finish the manual login first and then run:\n\n"
        f"\t!mkdir -p '{wandb_drive_netrc_path.parent}'\n"
        f"\t!cp {wandb_local_netrc_path} '{wandb_drive_netrc_path}'\n\n"
        f"Then that file will be used to login next time.\n"
    )

!wandb login

In [ ]:
#@title Clone `buganart/unagan` repo.
if IN_COLAB:
    %cd /content
    !git clone https://github.com/buganart/unagan
    clear_on_success("Repo cloned!")

In [ ]:
#@title Install dependencies
if IN_COLAB:
    %cd /content/unagan
    %pip install -q -r requirements.txt

clear_on_success("Dependencies installed!")

In [ ]:
#@title Configuration

#@markdown Wandb run id for `melgan` run.
melgan_run_id = "" #@param {type: "string"}
#a84z6f5s

#@markdown Wandb run id for `unagan` run.
unagan_run_id = "" #@param {type: "string"}
#2lttzztx

#@markdown If id is null, choose only one available run or loop through all runs.
random_pick_one_run = False #@param {type: "boolean"}

#@markdown whether to access API for the candidate list or use saved list.
check_API = False #@param {type: "boolean"}

#@markdown Duration of generate samples in seconds.
duration = 10 #@param {type: "integer"}

#@markdown Number of samples to generate.
num_samples = 1 #@param {type: "integer"}

#@markdown Random seed for sample generation.
seed = 123 #@param {type: "integer"}

#@markdown The path of the directory where the generated audio files are put.
output_dir = "/content/drive/MyDrive/IRCMS_GAN_collaborative_database/Experiments/colab-violingan/unagan-outputs" #@param {type:"string"}
output_dir = Path(output_dir)
output_dir.mkdir(parents=True, exist_ok=True)

unagan_run_id_record = ['3cs7wwdw', '13bfkv2p', '2xwq0s2t', '20te56qg', '3uy2er7w', '3i9ask6d', '2o3gbv1z', '1nckwb0l', 'vnuowoyp', '7ev2hycv', '30yoc8hd', '399pheul', '11qztpuj']
melgan_run_id_record = ['1sv05hdn', '2gvadb9k', 'a84z6f5s', '10evzcbq']

def get_run_candidate_list(model):
    if not check_API:
        if model == "unagan":
            return unagan_run_id_record
        else:
            return melgan_run_id_record

    import wandb
    import random
    import download_weights

    api = wandb.Api()
    all_runs = api.runs(f"demiurge/{model}")
    run_candidate_list = []
    for run in all_runs:
        #epoch test
        summary = run.summary._json_dict
        if "epoch" not in summary:
            continue
        if summary["epoch"] < 100:
            continue
        #TODO: 44.1kHz test
        #weight file test
        paths = download_weights.MODEL_PATHS[model]
        files = list(run.files())
        wandb_files_grouped_by_filename = download_weights.group_filenames(files)
        #check for all weight files
        failed = False
        for filename, _ in paths:
            #check weight file exists
            if filename not in wandb_files_grouped_by_filename:
                failed = True
                break
            # wandb_files = wandb_files_grouped_by_filename[filename]
            #check weight file duplicates
            # if len(wandb_files) > 1:
            #     failed = True
            #     break
        if failed:
            continue

        ### every test passed
        print(f"candidate id: {str(run.id)} found!")
        run_candidate_list.append(str(run.id))
    print(model + ": random select run id: found "+str(len(run_candidate_list))+" qualified runs.")
    print("candidate_list: "+str(run_candidate_list))
    #check list
    if len(run_candidate_list) == 0:
        raise Exception(model + ": No run in wandb is qualified.")
    return run_candidate_list

def random_select_id(model):
    #pick 1 from the run_candidate_list
    run_candidate_list = get_run_candidate_list(model)
    selected_run = random.choice(run_candidate_list)
    print(model + ": picked run_id: "+str(selected_run))
    return selected_run

def check_wandb_id(run_id, model):
    import re
    if run_id and not re.match(r"^[\da-z]{8}$", run_id):
        raise RuntimeError(
            "Run ID needs to be 8 characters long and contain only letters a-z and digits.\n"
            f"Got \"{run_id}\""
        )
    if not run_id:
        print(model+": run_id is empty. Pick random runs!")
        return random_select_id(model)
    else:
        return run_id
 

if random_pick_one_run or (unagan_run_id and melgan_run_id):
    unagan_run_id = check_wandb_id(unagan_run_id, "unagan")
    melgan_run_id = check_wandb_id(melgan_run_id, "melgan")

    multiple_runs_loop = False

    config = dict(
        melgan_run_id=melgan_run_id,
        unagan_run_id=unagan_run_id,
        duration=duration,
        num_samples=num_samples,
        seed=seed,
        output_dir=output_dir,
    )

    for k,v in config.items():
        print(f"=> {k:20}: {v}")
else:
    #one of the ids are None, and loop through the available run list
    multiple_runs_loop = True
    if not unagan_run_id:
        print("unagan_run_id is empty, find candidate_list!")
        unagan_run_id = get_run_candidate_list("unagan")
    else:
        unagan_run_id = [unagan_run_id]
    if not melgan_run_id:
        print("melgan_run_id is empty, find candidate_list!")
        melgan_run_id = get_run_candidate_list("melgan")
    else:
        melgan_run_id = [melgan_run_id]

In [ ]:
#@title Generate
import download_weights
import generate

def generate_sample(melgan_run_id, unagan_run_id):
    download_weights.main(
        melgan_run_id=melgan_run_id,
        unagan_run_id=unagan_run_id,
        model_dir=Path('models/custom')
    )

    generate.main(
        num_samples=num_samples, 
        gid=0,
        output_folder=output_dir,
        seed=seed,
        duration=duration,
        melgan_run_id=melgan_run_id,
        unagan_run_id=unagan_run_id,
    )
    print(f"melgan:{melgan_run_id}, unagan:{unagan_run_id}. Success! Samples saved to {output_dir}")

if multiple_runs_loop:
    for melgan_id in melgan_run_id:
        for unagan_id in unagan_run_id:
            try:
                generate_sample(melgan_id, unagan_id)
            except Exception as e:
                print(f"melgan:{melgan_id}, unagan:{unagan_id}. Failed")
                print("Error: " + str(e))
else:
    generate_sample(melgan_run_id, unagan_run_id)
